In [82]:
import numpy as np
import torch

In [83]:
def im2col(img, kernel_size, stride=1, padding=0):
    img_padded = torch.nn.functional.pad(img, (padding, padding, padding, padding), mode='constant', value=0)
    
    H, W = img_padded.shape
    out_h = (H - kernel_size) // stride + 1
    out_w = (W - kernel_size) // stride + 1
    
    cols = torch.zeros(kernel_size * kernel_size, out_h * out_w, device=img.device)
    
    col_idx = 0
    for i in range(0, H - kernel_size + 1, stride):
        for j in range(0, W - kernel_size + 1, stride):
            patch = img_padded[i:i+kernel_size, j:j+kernel_size]
            cols[:, col_idx] = patch.flatten()
            col_idx += 1
    
    return cols


In [84]:
def conv2d_im2col(img, kernel, stride=1, padding=0):
    cols = im2col(img, kernel.shape[1], stride, padding)
    
    kernel_flat = kernel.view(kernel.shape[0], -1)
    
    cols_t = cols.T
    
    out = torch.matmul(cols_t, kernel_flat.view(kernel_flat.shape[0], -1).T)
    
    return out.view(kernel.shape[0], cols.shape[1]).T

In [85]:
img = torch.tensor([[1, 2, 3, 4, 5],
                    [6, 7, 8, 9, 10],
                    [11, 12, 13, 14, 15],
                    [16, 17, 18, 19, 20],
                    [21, 22, 23, 24, 25]], dtype=torch.float32)

kernel = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]], dtype=torch.float32)

img = img.unsqueeze(0)
kernel = kernel.unsqueeze(0).unsqueeze(0)

conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=0, bias=False)
conv_layer.weight.data = kernel
output_torch = conv_layer(img)

output_im2col = conv2d_im2col(img.squeeze(0), kernel.squeeze(0), stride=1, padding=0)

print("Результат из torch.nn.Conv2d:")
print(output_torch)

print(output_im2col)

if torch.allclose(output_torch, output_im2col, atol=1e-5):
    print("\nРезультаты совпадают!")
else:
    print("\nРезультаты не совпадают.")

Результат из torch.nn.Conv2d:
tensor([[[-6., -6., -6.],
         [-6., -6., -6.],
         [-6., -6., -6.]]], grad_fn=<SqueezeBackward1>)
tensor([[-6.],
        [-6.],
        [-6.],
        [-6.],
        [-6.],
        [-6.],
        [-6.],
        [-6.],
        [-6.]])


RuntimeError: The size of tensor a (3) must match the size of tensor b (9) at non-singleton dimension 1